In [ ]:
# pip install transformers

     |████████████████████████████████| 3.8 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 34.2 MB/s 
     |████████████████████████████████| 895 kB 33.3 MB/s 
     |████████████████████████████████| 6.5 MB 45.0 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification

from utilize import get_tidyTweet, remove_punct

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# %tensorflow_version 2.x
# print('tensorflow version', tf.__version__)
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#     raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

tensorflow version 2.8.0
Found GPU at: /device:GPU:0


In [ ]:
path = 'data/'
files = os.listdir(path)
df = pd.concat([pd.read_csv(path+f) for f in files], ignore_index=True)
df = df.drop_duplicates()
df = df[df['language']=='en']
df = df.reset_index(drop=True)

In [ ]:
# clean data
df2 = df.copy()
df2 = df2[['date','tweet']]
df2['cleaned_tweet'] = df['tweet'].apply(get_tidyTweet).str.lower()
df2['cleaned_tweet'] = df2['cleaned_tweet'].apply(remove_punct).str.strip()

df2.head()

,date,tweet,cleaned_tweet
0,2022-03-07 14:31:38,"#Psaki: ""[U]nvaccinated #teenagers are 3X as ...",unvaccinated are 3x as likely to be hospitali...
1,2022-03-07 14:18:00,"Spoke with @EpochTimes for the second time, th...",spoke with for the second time the first bein...
2,2022-03-07 11:54:57,CCP's Plan 13579 - How to destroy Western coun...,ccp's plan 13579 how to destroy western count...
3,2022-03-07 09:28:25,@CawthornforNC Exactly! #China should be prose...,exactly should be prosecuted extensively but ...
4,2022-03-07 09:24:14,"And now it's ""official"": https://t.co/VEa84T...",and now it's official


In [ ]:
# create random label for dataset
rand_label = np.random.randint(2, size=len(df2))

# assign random ralel to df
df2['label'] = rand_label

print(df2.head(5))
print()
print(df2.loc[df2['label']==0].shape)
print(df2.loc[df2['label']==1].shape)


                  date                                              tweet  \
0  2022-03-07 14:31:38  #Psaki:  "[U]nvaccinated #teenagers are 3X as ...   
1  2022-03-07 14:18:00  Spoke with @EpochTimes for the second time, th...   
2  2022-03-07 11:54:57  CCP's Plan 13579 - How to destroy Western coun...   
3  2022-03-07 09:28:25  @CawthornforNC Exactly! #China should be prose...   
4  2022-03-07 09:24:14  And now it's "official":   https://t.co/VEa84T...   

                                       cleaned_tweet  label  
0  unvaccinated  are 3x as likely to be hospitali...      0  
1  spoke with  for the second time the first bein...      1  
2  ccp's plan 13579  how to destroy western count...      0  
3  exactly  should be prosecuted extensively but ...      1  
4                              and now it's official      1  

(12398, 4)
(12578, 4)


In [ ]:
X = df2['cleaned_tweet'].values
y = df2['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16733,)
(8243,)
(16733,)
(8243,)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# find max sequence length
max_seq_length = 0
for item in X:
    max_seq_length = max(max_seq_length, len(tokenizer.tokenize(item)))
print(max_seq_length)

72


In [ ]:
def gen_inputs_tensor(X, max_seq_length):
    input_ids = []
    attention_masks = []

    for item in X:
        encoded_dict = tokenizer.encode_plus(
            item,
            add_special_tokens=True,    # add [CLS] 101 & [SEP] 102
            max_length=max_seq_length+2,
            # pad_to_max_length=True,
            padding='max_length',       # add 0 up to max_seq_length
            return_attention_mask=True
        )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # convert to tensor
    input_ids = tf.convert_to_tensor(input_ids)
    attention_masks = tf.convert_to_tensor(attention_masks)

    return input_ids, attention_masks

In [ ]:
X_train_input, X_train_mask = gen_inputs_tensor(X_train, max_seq_length)
X_test_input, X_test_mask = gen_inputs_tensor(X_test, max_seq_length)

y_train_tensor = tf.convert_to_tensor(y_train)
y_test_tensor = tf.convert_to_tensor(y_test)

print(X_train_input.shape)
print(X_train_mask.shape)

print(X_test_input.shape)
print(X_test_mask.shape)

print(y_train_tensor.shape)
print(y_test_tensor.shape)

(16733, 74)
(16733, 74)
(8243, 74)
(8243, 74)
(16733,)
(8243,)


In [ ]:
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
log_dir = 'model/log'
path_save_model = 'model/bert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=path_save_model, 
                                                save_weights_only=True, 
                                                monitor='val_loss', 
                                                mode='min', 
                                                save_best_only=True), 
            tf.keras.callbacks.TensorBoard(log_dir=log_dir)]

In [ ]:
print(bert_model.summary())

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)

In [ ]:
bert_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
history = bert_model.fit(
    [X_train_input, X_train_mask],
    y_train_tensor,
    batch_size=32,
    epochs=1,
    validation_data=([X_test_input, X_test_mask], y_test_tensor),
    callbacks=callbacks
)

523/523 [==============================] - 809s 1s/step - loss: 0.6960 - accuracy: 0.5036 - val_loss: 0.6949 - val_accuracy: 0.4980
